In [1]:
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
import numpy as np
from scipy.stats import pearsonr


In [2]:
# Cargar datos
file_path = r'C:\Users\emisi\OneDrive\INCAE_Análisis de Datos, Innovación y Tecnología\PAIT\Nuestro Diario\Bases de Datos\Análisis de Bases de Datos\DatosNacionales.csv'
df = pd.read_csv(file_path, delimiter= ';')
df.head()

,id,fecha,day,month,year,region,titulo,subtitular,cant_notas,politica,...,publicidad,contraportada,modelo,palabras_titulo,palabras_st,envio_total,cobrable,devuelto,vendido,total_paginas
0,010222nacional,2/1/2022,Sunday,January,2022,nacional,celebracion lo lleva a la muerte,"ya no llego a darles el abrazo a sus papas, en...",4,0,...,0,modelo,1,6,13,94993,93185,11937,81248,36
1,010322nacional,3/1/2022,Monday,January,2022,nacional,campeon historico,los toros suman su primer titulo de liga nacional,5,0,...,1,deporte,1,2,9,110810,108409,6740,101669,34
2,010422nacional,4/1/2022,Tuesday,January,2022,nacional,arde ensambladora,se destruyen miles de motos en amatitlan,4,0,...,0,nacionales,1,2,7,97543,95151,8336,86815,32
3,010522nacional,5/1/2022,Wednesday,January,2022,nacional,balean albaniles,sicarios los esperaban en esquina del centro d...,3,0,...,1,modelo,1,2,10,96234,93842,14682,79160,32
4,010622nacional,6/1/2022,Thursday,January,2022,nacional,feliz dia de reyes,-,1,0,...,0,modelo,1,4,1,96254,93851,9796,84055,32


In [3]:
# Cargar el Dataset
file_path = "DatosNacionales.csv"  # Asegúrate de colocar la ruta correcta
df = pd.read_csv(file_path, delimiter=";")

# Eliminar las variables especificadas
columns_to_exclude = ["region", "titulo", "modelo_portada", "especial", "id", "fecha", "year", "subtitular", "cobrable", "envio_total", "devuelto"]
df = df.drop(columns=columns_to_exclude, errors="ignore")

# Lista de variables categóricas que deben ser convertidas a dummies
categorical_vars = ["day", "month", "contraportada"]

# Verificar qué variables categóricas existen en el dataset
existing_categorical_vars = [col for col in categorical_vars if col in df.columns]

# Convertir variables categóricas en dummies
df = pd.get_dummies(df, columns=existing_categorical_vars, drop_first=True)

# Asegurar que todas las variables sean numéricas
df = df.apply(pd.to_numeric, errors="coerce")

# Definir las variables predictoras (X) excluyendo "vendido"
X = df.drop(columns=["vendido"])

# Variable objetivo (y)
y = df["vendido"]

# Llenar valores NaN para evitar errores en la regresión
X = X.fillna(X.median())  # Usando la mediana
y = y.fillna(y.median())  # Usando la mediana

# Convertir todas las variables a tipo float
X = X.astype(float)
y = y.astype(float)

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir la cuadrícula de hiperparámetros: n_estimators y max_depth
valores_n_estimators = [20, 40, 60, 80, 100]
valores_max_depth = [5, 10, 15, 25]

# Inicializar lista para almacenar resultados
resultados = []

# Iterar sobre las combinaciones de n_estimators y max_depth
for n_estimators in valores_n_estimators:
    for max_depth in valores_max_depth:
        print(f"Entrenando Gradient Boosted Trees con n_estimators={n_estimators}, max_depth={max_depth}")
        
        # Crear y ajustar el modelo
        gb_modelo = GradientBoostingRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        gb_modelo.fit(X_train, y_train)
        
        # Generar predicciones
        y_pred = gb_modelo.predict(X_test)
        
        # 1. Root Mean Squared Error (RMSE)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        
        # 2. Absolute Error
        absolute_error = mean_absolute_error(y_test, y_pred)
        
        # 3. Relative Error (Lenient)
        relative_error = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
        
        # 4. Squared Error
        squared_error = np.mean((y_test - y_pred) ** 2)
        
        # 5. Correlation Coefficient (R)
        correlation, _ = pearsonr(y_test, y_pred)
        
        # Almacenar los resultados en la lista
        resultados.append({
            "Number of Trees": n_estimators,
            "Maximal Depth": max_depth,
            "RMSE": rmse,
            "Absolute Error": absolute_error,
            "Relative Error (%)": relative_error,
            "Squared Error": squared_error,
            "Correlation Coefficient": correlation
        })

# Crear un DataFrame para visualizar los resultados
df_resultados = pd.DataFrame(resultados)

# Ordenar el DataFrame por las métricas en orden ascendente o descendente según sea necesario
df_resultados_ranked_rmse = df_resultados.sort_values(by="RMSE", ascending=True)  # Mejor RMSE al principio
df_resultados_ranked_absolute_error = df_resultados.sort_values(by="Absolute Error", ascending=True)  # Mejor Error Absoluto al principio
df_resultados_ranked_r2 = df_resultados.sort_values(by="Correlation Coefficient", ascending=False)  # Mejor R² al principio

# Mostrar los resultados ordenados en ranking para cada métrica
print("\nRanking de modelos por RMSE:")
print(df_resultados_ranked_rmse)

print("\nRanking de modelos por Error Absoluto:")
print(df_resultados_ranked_absolute_error)

print("\nRanking de modelos por R²:")
print(df_resultados_ranked_r2)

Entrenando Gradient Boosted Trees con n_estimators=20, max_depth=5
Entrenando Gradient Boosted Trees con n_estimators=20, max_depth=10
Entrenando Gradient Boosted Trees con n_estimators=20, max_depth=15
Entrenando Gradient Boosted Trees con n_estimators=20, max_depth=25
Entrenando Gradient Boosted Trees con n_estimators=40, max_depth=5
Entrenando Gradient Boosted Trees con n_estimators=40, max_depth=10
Entrenando Gradient Boosted Trees con n_estimators=40, max_depth=15
Entrenando Gradient Boosted Trees con n_estimators=40, max_depth=25
Entrenando Gradient Boosted Trees con n_estimators=60, max_depth=5
Entrenando Gradient Boosted Trees con n_estimators=60, max_depth=10
Entrenando Gradient Boosted Trees con n_estimators=60, max_depth=15
Entrenando Gradient Boosted Trees con n_estimators=60, max_depth=25
Entrenando Gradient Boosted Trees con n_estimators=80, max_depth=5
Entrenando Gradient Boosted Trees con n_estimators=80, max_depth=10
Entrenando Gradient Boosted Trees con n_estimators=8